In [248]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import json
load_dotenv()

True

In [247]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name='stock-advise',
    dimension=1536,
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1')
)



In [323]:
data = json.load(open("/Users/ceo/Desktop/headstarter_SWE/Final_Project/Buddyfin-AI/src/data/reviews.json"))

In [324]:
new_data = []
client = OpenAI()

for stocks in data['best_stocks']:
    input_text = f"{stocks['company']} {stocks['ticker']}"
    response = client.embeddings.create(
        input = input_text,
        model = 'text-embedding-3-small',
    )

    embedding = response.data[0].embedding
    new_data.append({
        "values": embedding,
        'id': stocks['company'],
        'metadata':{
            'ticker': stocks['ticker'],
            'analysis': stocks['analysts_recommendation'],
            'ratings': stocks['ratings'],
        }
        }
    )

for stocks in data['poor_stocks']:
    input_text = f"{stocks['company']} {stocks['ticker']}"
    response = client.embeddings.create(
        input = input_text,
        model = 'text-embedding-3-small',
    )

    embedding = response.data[0].embedding
    new_data.append({
        "values": embedding,
        'id': stocks['company'],
        'metadata':{
            'ticker': stocks['ticker'],
            'analysis': stocks['analysts_recommendation'],
            'ratings': stocks['ratings'],
        }
        }
    )



In [325]:
index = pc.Index('stock-advise')
index.upsert(
    vectors=new_data,
    namespace='ns1'
)

{'values': [-0.02458380162715912,
  0.011517300270497799,
  0.01193198561668396,
  0.0032842301297932863,
  0.007382182404398918,
  0.013340351171791553,
  -0.022768575698137283,
  0.01131386961787939,
  0.010922656394541264,
  -0.04516158625483513,
  0.02508455328643322,
  0.001441618544049561,
  0.007030090782791376,
  -0.040905192494392395,
  0.059213943779468536,
  -0.007691240403801203,
  0.0238952673971653,
  -0.02004573494195938,
  -0.0605597160756588,
  0.07536320388317108,
  0.0019110736902803183,
  0.030702367424964905,
  -0.048854634165763855,
  -0.0046789031475782394,
  -0.033487800508737564,
  0.0025272336788475513,
  -0.012002403847873211,
  -0.02419258840382099,
  0.03840143233537674,
  0.0031786025501787663,
  0.045411963015794754,
  -0.036210641264915466,
  -0.0031179646030068398,
  -0.03758770972490311,
  -0.0070222667418420315,
  0.010523620061576366,
  -0.01123562641441822,
  0.05070115625858307,
  -0.022768575698137283,
  0.05148358270525932,
  -0.00180838035885244